# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,27.99,89,40,5.66,CC,1718578894
1,1,albany,42.6001,-73.9662,24.26,52,96,0.45,US,1718578847
2,2,rio grande,-32.0350,-52.0986,17.06,90,100,7.22,BR,1718578895
3,3,longyearbyen,78.2186,15.6401,5.91,75,75,7.20,SJ,1718578895
4,4,crane,31.3974,-102.3501,37.64,23,0,10.77,US,1718578895


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = True,
    color = "City",
    alpha = 0.6,
    hover_cols=["City", "Humidity"],
    cmap = "Category20"

)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
temp_min = 21
temp_max = 27
wind_speed = 4.5
cloudiness = 0

narrow_df = city_data_df[
    (city_data_df['Max Temp'] >= temp_min) & 
    (city_data_df['Max Temp'] <= temp_max) &
    (city_data_df['Wind Speed'] <= wind_speed) &
    (city_data_df['Cloudiness'] == cloudiness)
]

# Drop any rows with null values
narrow_df = narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,vilhena,-12.7406,-60.1458,26.12,61,0,2.06,BR,1718578898
39,39,bethel,41.3712,-73.4140,24.24,57,0,3.09,US,1718578691
232,232,jaen,38.0000,-3.5000,23.30,23,0,2.09,ES,1718578842
296,296,morondava,-20.2833,44.2833,21.57,60,0,2.63,MG,1718578989
310,310,la foa,-21.7108,165.8276,23.59,54,0,3.08,NC,1718578994


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,vilhena,BR,-12.7406,-60.1458,61,
39,bethel,US,41.3712,-73.4140,57,
232,jaen,ES,38.0000,-3.5000,23,
296,morondava,MG,-20.2833,44.2833,60,
310,la foa,NC,-21.7108,165.8276,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
  "categories": "accommodation.hotel",
  "limit": 1,
  "apiKey": geoapify_key
  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rio grande - nearest hotel: Hotel Vila Moura Executivo
longyearbyen - nearest hotel: The Vault
alice springs - nearest hotel: Aurora Alice Springs
gadzhiyevo - nearest hotel: Оазис
cles - nearest hotel: Hotel Antica Trattoria
east london - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
chonchi - nearest hotel: Hotel Huildín
kodiak - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
kharp - nearest hotel: Собь
demyansk - nearest hotel: Гостиница Демянская
ulladulla - nearest hotel: Hotel Marlin
udachny - nearest hotel: Вилюй
waitangi - nearest hotel: Hotel Chathams
ketchikan - nearest hotel: Cape Fox Lodge
nymburk - nearest hotel: Grand hotel Postřižiny
kodinsk - nearest hotel: Ангара
uray - nearest hotel: Турсунт
saint lucia - nearest hotel: Jephson Hotel
edinburgh of the seven seas - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
thompson - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
2,rio grande,BR,-32.0350,-52.0986,90,Hotel Vila Moura Executivo
3,longyearbyen,SJ,78.2186,15.6401,75,The Vault
8,alice springs,AU,-23.7000,133.8833,93,Aurora Alice Springs
11,gadzhiyevo,RU,69.2551,33.3362,73,Оазис
12,cles,IT,46.3667,11.0333,100,Hotel Antica Trattoria
...,...,...,...,...,...,...
567,fort mcmurray,CA,56.7268,-111.3810,40,Nomad Hotel and Suites
569,collie,AU,-33.3667,116.1500,84,Federal Hotel
570,huy-neinstedt,DE,51.9667,10.9167,87,Hotel & Spa Wasserschloss Westerburg
572,novodvinsk,RU,64.4165,40.8122,71,Уют


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    size = "Humidity",
    color = "City",
    alpha = 0.6,
    cmap = "Category20",
    hover_cols = ['Hotel Name', 'City']

)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)